This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [138]:
base_directory = '/Volumes/OneTouch/datasets/adni/metadata/updated_master_list/master.csv'
file_pattern = None #'*.nii.gz'
file_column = 'csf_paths'

In [139]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
dv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=file_pattern, file_column=file_column, process_special_values=True).run()
dv_df

Attempting to import from: master.csv


,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0295/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0295_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0413/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0413_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0559/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0559_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0619/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0619_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0685/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0685_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0729/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0729_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0782/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0782_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0816/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0816_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0938/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0938_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0954/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0954_cerebrospinal_fluid.nii.gz,...,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4066/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4066_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4100/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4100_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4187/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4187_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4255/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4255_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4292/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4292_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4365/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4365_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4376/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4376_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4377/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4377_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4420/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4420_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4764/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4764_cerebrospinal_fluid.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

This Should Often Be Left Default

In [140]:
preceding_id = '/sub-'
proceeding_id = ''

In [141]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
# dv_df = GiiNiiFileImport.splice_colnames(dv_df, preceding_id, proceeding_id)
dv_df

,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0295/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0295_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0413/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0413_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0559/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0559_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0619/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0619_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0685/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0685_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0729/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0729_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0782/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0782_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0816/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0816_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0938/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0938_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0954/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0954_cerebrospinal_fluid.nii.gz,...,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4066/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4066_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4100/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4100_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4187/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4187_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4255/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4255_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4292/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4292_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4365/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4365_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4376/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4376_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4377/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4377_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4420/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4420_cerebrospinal_fluid.nii.gz,/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4764/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4764_cerebrospinal_fluid.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# Load Map to Check Damage Within

Import Region of Interest Masks

In [142]:
base_directory = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined'
shared_glob_pattern = '*.nii.gz'

In [143]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
iv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=shared_glob_pattern).run()
iv_df

Attempting to import from: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/*.nii.gz


,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Fusiform_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Temporal_Pole_Mid_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Occipital_Sup_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Postcentral_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Cerebelum_Crus2_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Temporal_Inf_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Rolandic_Oper_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Cerebelum_9_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Rectus_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Temporal_Sup_.nii.gz,...,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Frontal_Sup_Medial_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Vermis_8_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Vermis_4_5_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Thalamus_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/OFCant_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Vermis_9_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Frontal_Mid_2_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Frontal_Inf_Orb_2_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Frontal_Inf_Oper_.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/AAL/combined/Cingulate_Ant_.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Extract Damage Scores Per Region of Interest

In [144]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [145]:
from calvin_utils.nifti_utils.damage_score_utils import DamageScorer
damage_scorer = DamageScorer(mask_path, dv_df, iv_df)
dmg_df = damage_scorer.calculate_damage_scores(['sum'])
dmg_df = damage_scorer.sort_dataframes_by_index(dmg_df)
dmg_df

,Fusiform__sum,Temporal_Pole_Mid__sum,Occipital_Sup__sum,Postcentral__sum,Cerebelum_Crus2__sum,Temporal_Inf__sum,Rolandic_Oper__sum,Cerebelum_9__sum,Rectus__sum,Temporal_Sup__sum,...,Frontal_Sup_Medial__sum,Vermis_8__sum,Vermis_4_5__sum,Thalamus__sum,OFCant__sum,Vermis_9__sum,Frontal_Mid_2__sum,Frontal_Inf_Orb_2__sum,Frontal_Inf_Oper__sum,Cingulate_Ant__sum
path,,,,,,,,,,,,,,,,,,,,,
/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0295/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0295_cerebrospinal_fluid.nii.gz,1187.716589,166.564130,-262.611913,29.964465,375.961496,1008.910840,79.898858,145.816903,43.482387,300.392584,...,416.403390,63.813063,-102.714406,-34.033036,311.472983,-17.366648,412.100553,271.280578,-209.348924,721.253283
/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0413/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0413_cerebrospinal_fluid.nii.gz,-334.480330,-166.825894,-345.391224,-1457.195447,-354.068076,-676.690654,-150.245099,309.116445,236.896343,-294.768421,...,-128.469319,-4.394796,-162.021067,-12.026893,74.537601,32.158485,-176.736374,221.634925,27.725340,-144.223406
/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0559/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0559_cerebrospinal_fluid.nii.gz,634.731834,29.527397,283.255447,286.840979,2030.057216,649.522957,236.212388,966.276307,191.964231,566.832513,...,-139.028697,85.825515,301.719512,11.963460,137.553253,260.487930,512.167687,33.669479,-136.530317,514.695790
/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0619/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0619_cerebrospinal_fluid.nii.gz,3431.119744,322.385677,-61.484594,1549.857107,385.633090,3096.784957,2201.078088,652.147525,968.732434,3646.901931,...,4288.974421,23.682282,821.513989,1557.604162,379.665564,71.098365,2732.569412,761.678787,268.375673,2461.032282
/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-002_S_0685/ses-01/unthresholded_tissue_segment_z_scores/sub-002_S_0685_cerebrospinal_fluid.nii.gz,1480.587238,161.511175,1024.541514,418.981130,527.520243,537.233761,583.908171,150.321768,-11.793763,1086.770586,...,-779.147004,40.312821,238.997306,140.930352,45.850147,1.430872,260.396969,449.751202,233.586464,-179.493887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4365/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4365_cerebrospinal_fluid.nii.gz,2035.756331,-273.271051,647.112602,1010.628868,1461.980145,2094.209771,987.215441,739.941920,381.229303,2232.792746,...,1349.981648,90.059777,71.323542,377.787769,70.376348,39.203013,659.427149,296.715059,961.520171,1183.375183
/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4376/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4376_cerebrospinal_fluid.nii.gz,365.417725,-112.260598,643.282858,429.157348,422.834811,903.401360,80.699803,-69.394319,-101.258247,663.238853,...,-541.197370,-2.409408,80.759112,-42.566077,17.525167,-11.172219,722.453327,108.787385,250.194350,-181.363419
/Volumes/OneTouch/datasets/adni/neuroimaging/all_patients_atrophy_seeds/sub-941_S_4377/ses-01/unthresholded_tissue_segment_z_scores/sub-941_S_4377_cerebrospinal_fluid.nii.gz,305.698511,157.416271,563.908837,-248.410778,571.098654,-697.477657,-368.052217,-204.805656,-10.449628,-811.888424,...,638.456015,-28.356295,879.722857,-110.844677,-223.241386,-28.787684,-1056.194613,-123.845968,0.955352,31.383928


Rename Column in Damage DF

In [146]:
dmg_df = dmg_df.rename_axis('roi_path').reset_index()

In [147]:
dmg_df = dmg_df.rename(columns={dmg_df.columns[0]: 'csf_paths'})

In [148]:
dmg_df.columns = [dmg_df.columns[0]] + [f"{col}_csf" for col in dmg_df.columns[1:]]

In [149]:
dmg_df

,csf_paths,Fusiform__sum_csf,Temporal_Pole_Mid__sum_csf,Occipital_Sup__sum_csf,Postcentral__sum_csf,Cerebelum_Crus2__sum_csf,Temporal_Inf__sum_csf,Rolandic_Oper__sum_csf,Cerebelum_9__sum_csf,Rectus__sum_csf,...,Frontal_Sup_Medial__sum_csf,Vermis_8__sum_csf,Vermis_4_5__sum_csf,Thalamus__sum_csf,OFCant__sum_csf,Vermis_9__sum_csf,Frontal_Mid_2__sum_csf,Frontal_Inf_Orb_2__sum_csf,Frontal_Inf_Oper__sum_csf,Cingulate_Ant__sum_csf
0,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,1187.716589,166.564130,-262.611913,29.964465,375.961496,1008.910840,79.898858,145.816903,43.482387,...,416.403390,63.813063,-102.714406,-34.033036,311.472983,-17.366648,412.100553,271.280578,-209.348924,721.253283
1,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,-334.480330,-166.825894,-345.391224,-1457.195447,-354.068076,-676.690654,-150.245099,309.116445,236.896343,...,-128.469319,-4.394796,-162.021067,-12.026893,74.537601,32.158485,-176.736374,221.634925,27.725340,-144.223406
2,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,634.731834,29.527397,283.255447,286.840979,2030.057216,649.522957,236.212388,966.276307,191.964231,...,-139.028697,85.825515,301.719512,11.963460,137.553253,260.487930,512.167687,33.669479,-136.530317,514.695790
3,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,3431.119744,322.385677,-61.484594,1549.857107,385.633090,3096.784957,2201.078088,652.147525,968.732434,...,4288.974421,23.682282,821.513989,1557.604162,379.665564,71.098365,2732.569412,761.678787,268.375673,2461.032282
4,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,1480.587238,161.511175,1024.541514,418.981130,527.520243,537.233761,583.908171,150.321768,-11.793763,...,-779.147004,40.312821,238.997306,140.930352,45.850147,1.430872,260.396969,449.751202,233.586464,-179.493887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,2035.756331,-273.271051,647.112602,1010.628868,1461.980145,2094.209771,987.215441,739.941920,381.229303,...,1349.981648,90.059777,71.323542,377.787769,70.376348,39.203013,659.427149,296.715059,961.520171,1183.375183
1302,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,365.417725,-112.260598,643.282858,429.157348,422.834811,903.401360,80.699803,-69.394319,-101.258247,...,-541.197370,-2.409408,80.759112,-42.566077,17.525167,-11.172219,722.453327,108.787385,250.194350,-181.363419
1303,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,305.698511,157.416271,563.908837,-248.410778,571.098654,-697.477657,-368.052217,-204.805656,-10.449628,...,638.456015,-28.356295,879.722857,-110.844677,-223.241386,-28.787684,-1056.194613,-123.845968,0.955352,31.383928
1304,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,588.632597,220.003687,401.180579,978.135831,-157.876675,1078.158040,1262.461483,268.484577,113.812290,...,1269.655256,-17.575315,-136.741322,59.814035,285.885410,-21.225991,1618.910134,131.868215,28.298133,1162.736856


Save the Results

In [150]:
# out_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/metadata/master_list2.csv'

In [151]:
# dmg_df.to_csv(out_path)

Save Results into Master CSV

In [152]:
master_path = '/Volumes/OneTouch/datasets/adni/metadata/updated_master_list/master.csv'
sheet = None

In [153]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
cal_palm = CalvinStatsmodelsPalm(input_csv_path=master_path, output_dir=None, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,subject,roi_file_path,conn_file_path,subu,Age,Male,Female,CSF_Cerebellum,CSF_Subcortex,CSF_MTL,...,Frontal_Sup_Medial__sum_gm,Vermis_8__sum_gm,Vermis_4_5__sum_gm,Thalamus__sum_gm,OFCant__sum_gm,Vermis_9__sum_gm,Frontal_Mid_2__sum_gm,Frontal_Inf_Orb_2__sum_gm,Frontal_Inf_Oper__sum_gm,Cingulate_Ant__sum_gm
0,002_S_0295,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0295,84.898630,1.0,0.0,-26319.38152,373.297842,-746.688684,...,-1459.277445,-32.741996,108.002663,-488.425315,-537.697264,16.492348,-2049.667647,110.933831,-469.235994,996.049458
1,002_S_0413,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0413,76.397260,0.0,1.0,-13670.88871,5009.504947,-449.838607,...,-181.217483,-29.428614,164.379278,-765.321262,689.259791,-88.893794,815.593236,237.845327,973.343977,-184.332195
2,002_S_0559,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0559,79.410959,1.0,0.0,-38702.13792,152.392985,-754.706246,...,-1333.182087,-109.647838,-249.407795,-3185.530540,-58.912668,-4.102355,-1461.804968,-254.480427,-965.509998,-137.063169
3,002_S_0619,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0619,77.512329,1.0,0.0,-35472.21632,-23876.106190,-1947.061673,...,-1764.835262,-116.257679,-99.573571,-4515.736978,-317.049204,-69.425741,-605.174374,-635.779999,-1153.279183,-1110.102469
4,002_S_0685,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0685,89.698630,0.0,1.0,-31092.93296,-22643.911490,-1014.786011,...,-905.685912,177.801752,-1.708630,-579.734070,-113.470557,204.759017,-495.643974,-438.856293,-114.393415,-903.974971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,100_S_1154,NaN,NaN,100uSu1154,76.646575,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1385,100_S_1226,NaN,NaN,100uSu1226,82.660274,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1386,100_S_1286,NaN,NaN,100uSu1286,75.638356,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1387,121_S_1322,NaN,NaN,121uSu1322,71.750685,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
# Perform an outer join on dmg_df and data_df using the 'path' column
merged_df = data_df.merge(dmg_df, on='csf_paths', how='outer')
merged_df

,subject,roi_file_path,conn_file_path,subu,Age,Male,Female,CSF_Cerebellum,CSF_Subcortex,CSF_MTL,...,Frontal_Sup_Medial__sum_csf,Vermis_8__sum_csf,Vermis_4_5__sum_csf,Thalamus__sum_csf,OFCant__sum_csf,Vermis_9__sum_csf,Frontal_Mid_2__sum_csf,Frontal_Inf_Orb_2__sum_csf,Frontal_Inf_Oper__sum_csf,Cingulate_Ant__sum_csf
0,002_S_0295,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0295,84.898630,1.0,0.0,-26319.38152,373.297842,-746.688684,...,416.403390,63.813063,-102.714406,-34.033036,311.472983,-17.366648,412.100553,271.280578,-209.348924,721.253283
1,002_S_0413,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0413,76.397260,0.0,1.0,-13670.88871,5009.504947,-449.838607,...,-128.469319,-4.394796,-162.021067,-12.026893,74.537601,32.158485,-176.736374,221.634925,27.725340,-144.223406
2,002_S_0559,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0559,79.410959,1.0,0.0,-38702.13792,152.392985,-754.706246,...,-139.028697,85.825515,301.719512,11.963460,137.553253,260.487930,512.167687,33.669479,-136.530317,514.695790
3,002_S_0619,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0619,77.512329,1.0,0.0,-35472.21632,-23876.106190,-1947.061673,...,4288.974421,23.682282,821.513989,1557.604162,379.665564,71.098365,2732.569412,761.678787,268.375673,2461.032282
4,002_S_0685,NaN,/Volumes/Expansion/datasets/adni/neuroimaging/...,002uSu0685,89.698630,0.0,1.0,-31092.93296,-22643.911490,-1014.786011,...,-779.147004,40.312821,238.997306,140.930352,45.850147,1.430872,260.396969,449.751202,233.586464,-179.493887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,100_S_1154,NaN,NaN,100uSu1154,76.646575,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1385,100_S_1226,NaN,NaN,100uSu1226,82.660274,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1386,100_S_1286,NaN,NaN,100uSu1286,75.638356,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1387,121_S_1322,NaN,NaN,121uSu1322,71.750685,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
merged_df.to_csv(master_path, index=False)

In [156]:
master_path

'/Volumes/OneTouch/datasets/adni/metadata/updated_master_list/master.csv'

All done. Enjoy your analyses. 

--Calvin 